In [ ]:
# установка внешних программ и библиотек на виртуальную машину
!git clone --recurse-submodules https://github.com/KarlsruheMIS/KaMIS.git
%cd KaMIS
!./compile_withcmake.sh

%cd ..


!pip3 install python-sat
#!sudo apt install minisat
!git clone https://github.com/arminbiere/kissat.git
%cd kissat
!./configure
!make

%cd ..

!cp KaMIS/deploy/redumis redumis
!cp KaMIS/deploy/online_mis online_mis
!cp kissat/build/kissat kissat1



Cloning into 'KaMIS'...
remote: Enumerating objects: 1449, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 1449 (delta 27), reused 21 (delta 19), pack-reused 1399
Receiving objects: 100% (1449/1449), 12.76 MiB | 10.95 MiB/s, done.
Resolving deltas: 100% (818/818), done.
/content/KaMIS
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test COMPILER_SUPPORTS_FUNROLL_LOOPS
--

In [1]:
# функции из библиотеки
# раскраска графа в формате  DIMACS, преобразование в CNF

import math
import numpy as np
from numpy import linalg as LA
import networkx as nx

#import matplotlib.pylab as plt

#from google.colab import files 
#files.upload() 
#files.download("file.txt")

#https://pysathq.github.io/
#!pip install python-sat
#import pysat
from pysat.formula import CNF
from pysat.solvers import *

import os
import sys
import random
from zipfile import ZipFile
#from tqdm import tqdm_notebook as tqdm
from itertools import combinations, permutations
import glob

out_dir = ""

class Utils:
    
    def read_dimacs_graph(file = 'graph.col'):
        
        if not (os.path.exists(file) and os.path.getsize(file) > 0):        
            raise Exception("File " + file + " not found")
        
        nodes = []    
        edges = []
        labels = []
        got_labels = False
        nnodes = nedges = 0
        
        with open(file, 'r') as f:
            for line in f:
                line = [l.strip() for l in line.split(' ')]
                if line[0] == 'c': #comment
                    continue
                elif line[0] == 'p': #problem
                    nnodes = int(line[2])
                    nedges = int(line[3])
                    nodes = list(range(1, nnodes + 1))
                    labels = [0] * nnodes
                elif line[0] == 'e': #edge
                    edges.append((int(line[1]), int(line[2])))
                elif line[0] == 'l':
                    labels[int(line[1]) - 1] = int(line[2])
                    got_labels = True

        if got_labels:        
            nodes = [(n, {'c' : l}) for n, l in zip(nodes, labels)]

        g = nx.Graph()
        g.add_nodes_from(nodes)
        g.add_edges_from(edges)
        return g

    def write_dimacs_graph(file = 'graph.col', g = nx.Graph(), comments = []):
        with open(file, 'w') as f:
            for c in comments:
                f.write("c " + c + "\n")
            f.write("p EDGE {} {}\n".format(g.number_of_nodes(), g.number_of_edges()))
            for u, v in g.edges():
                f.write("e {} {}\n".format(u, v))
            for node in g.nodes():
                if 'c' in g.node[node]:
                    f.write("l {} {}\n".format(node, g.node[node]['c']))

    def draw_with_colors(g = nx.Graph()):
        color_map = []
        for node in g.nodes():
            if 'c' in g.node[node]:
                color_map.append(g.node[node]['c'] * 10)            
        nx.draw(g, pos = nx.spring_layout(g, scale=2), node_color=color_map, with_labels=True, cmap = plt.cm.jet)

    def write_proof(file = "proof.txt", proof = []):
        with open(file, 'w') as f:
            for p in proof:
                f.write("%s\n" % str(p))

    def zip_files(file = "archive.zip", files = []):
        with ZipFile(file, 'w') as archive:
            for f in set(files):                
                if not (os.path.exists(file) and os.path.getsize(file) > 0):        
                    raise Exception("File " + file + " not found")
                archive.write(f)
                
def find_triangle(g = nx.Graph()):
    for a in g:
        for b, c in combinations(g[a], 2):
            if b in g[c]:
                return [a, b, c]
    return []
             
    #return set(frozenset([a, b, c]) for a in g for b, c in combinations(g[a], 2) if b in g[c])


def find_isolates(g = nx.Graph()):
    isolates = []
    for n in g:
        if g.degree(n) == 0:
            isolates.append(n)
    return isolates

class ColMap:
    
    def __init__(self, g = nx.Graph(), ncolors = 40):
        
        self.ncolors = ncolors
        self.cmap = dict()
        self.cunmap = dict()
    
        i = 1
        for node in g.nodes():
            for color in range(1, ncolors + 1):            
                self.cmap[(node, color)] = i
                self.cunmap[i] = (node, color)
                i += 1    

    def enc(self, node, color):
        return self.cmap[(node, color)]

    def dec(self, node_color):
        return self.cunmap[node_color]

class ColSAT:
   
    def __init__(self, g = nx.Graph(), ncolors = 10):
        
        self.ncolors = ncolors
        self.g = g.copy()
        self.cmap = ColMap(g, ncolors)        

    def check_coloring(self):
        for n1, n2 in self.g.edges():
            if 'c' not in self.g.node[n1] or 'c' not in self.g.node[n2]:
                return False
            if self.g.node[n1]['c'] == self.g.node[n2]['c']:
                return False
        return True
    
    def apply_model(self):
        
        check = set()
        for var in self.model[self.model > 0]:        
            node, color = self.cmap.dec(var)
            self.g.node[node]['c'] = color
            if (node, color) in check:
                raise Exception("Two colors for one node???")
            else:
                check.add((node, color))
        
        self.colored = self.check_coloring()
        
        if self.colored != self.solved:
            raise Exception("Something went wrong!")
        
        return self.colored
        
    def build_cnf(self, ):
        
        self.formula = CNF()
        colors = list(range(1, self.ncolors + 1))    

        maxc = -1        # condition k<12 is not for general case
        k = 0
        for clique in nx.find_cliques(self.g):
            if len(clique)>maxc:
                maxclique = clique.copy()  
                maxc = len(clique)
            if k<12:
                k += 1
                continue
           
            col = 1
            if len(maxclique)>self.ncolors:
                print("Size of the maximal clique is {}".format(len(maxclique)))
                raise Exception("Size of the maximal clique is {} > {}".format(len(maxclique),ncolors))
            for v in maxclique:
                self.formula.append([self.cmap.enc(v, col)])
                col += 1
            break

        for n1, n2 in self.g.edges():
            for c in colors:            
                self.formula.append([-self.cmap.enc(n1, c), -self.cmap.enc(n2, c)])


        for n in self.g.nodes():
            #if not n in specials:
            self.formula.append([self.cmap.enc(n, c) for c in colors])
            #for c1 in colors:
            #    for c2 in colors:
            #        if c1 < c2:
            #            self.formula.append([-self.cmap.enc(n, c1), -self.cmap.enc(n, c2)])
        
        return self.formula
    
    def solve_cnf(self, solver = ''):
        
        triangle = find_triangle(self.g)
        assumptions = []
        if len(triangle) > 0:            
            assumptions = [self.cmap.enc(triangle[0], 1), self.cmap.enc(triangle[1], 2), self.cmap.enc(triangle[2], 3)]
            
        #Glucose3, Glucose4, Lingeling, MapleChrono, MapleCM, Maplesat, Minisat22, MinisatGH
        #with Glucose4(bootstrap_with=self.formula.clauses, with_proof=True) as ms:        
        with Lingeling(bootstrap_with=self.formula.clauses) as ms:
            self.solved = ms.solve(assumptions=assumptions)
            if self.solved:
                self.model = np.array(ms.get_model())
                self.apply_model()
            else:                
                self.proof = []#ms.get_proof()
                self.colored = False
                
        return self.solved






In [2]:
# выбор графа на вершинах гиперкуба (покрышки), удаление независимого множества,
# проверка существования раскраски

import networkx as nx

def dist(a,b):
  return sum([int(c) for c in bin(a^b).split('b')[1]])

def bin_graph(n, k, trim = []):   # третий параметр - множество вершин
  g = nx.Graph()                  # по k-окрестностям которых происходит отсечение    
  for i in range(0,2**n):
    if dist(0,i)%2==1:
      continue
    flag = True
    for t in trim:
      if dist(t,i)>k:
        flag = False
        break
    if flag:
      g.add_node(i)
  for i in list(g.nodes()):
    for j in g.nodes():
      if (i<j) and (dist(i,j)==k):
        g.add_edge(i,j)
  return nx.convert_node_labels_to_integers(g)

def small_graph(n, k, dists = [2,4,8]):   # третий параметр - множество вершин
  g = nx.Graph()                  # по k-окрестностям которых происходит отсечение    
  for i in range(0,2**n):
    if dist(0,i) not in dists:
      continue
    g.add_node(i)
  for i in list(g.nodes()):
    for j in g.nodes():
      if (i<j) and (dist(i,j)==k):
        g.add_edge(i,j)
  return nx.convert_node_labels_to_integers(g)

def bin_graph1(n, k, trim = []):
  g = nx.Graph()
  for i in range(0,2**n):
    if dist(0,i)%2==1:
      continue
    flag = True
    if dist(0,i)>k:
      flag = False
      
    if flag or i in set(trim):
      g.add_node(i)
  for i in list(g.nodes()):
    for j in g.nodes():
      if (i<j) and (dist(i,j)==k):
        g.add_edge(i,j)
  return nx.convert_node_labels_to_integers(g)


def r_graph(n, k):      # граф запретов,
  g = nx.Graph()        # ребрами соединены вершины на расстоянии >k
  for i in range(0,2**n):
    g.add_node(i)
  for i in list(g.nodes()):
    for j in g.nodes():
      if (i<j) and (dist(i,j)>k):
        g.add_edge(i,j)
  return g

def write_metis(g0,fname):    # запись графа во входном формате KaMIS
  g = nx.convert_node_labels_to_integers(g0)
  with open(fname,"w") as f:
    f.write("{} {}\n".format(g.number_of_nodes(),g.number_of_edges()))
    for v in range(g.number_of_nodes()):
      f.write(" ".join([str(i+1) for i in sorted(list(g[v]))])+"\n")    

def write_cnf(g,ncolors,filename):  # запись КНФ для SAT-решателя    
  problem = ColSAT(g, ncolors)
  problem.build_cnf().to_file(filename)

def select_mis(g,filename):  # возвращает независимое множество
  g1 = g.copy()
  with open(filename,"r") as f:
    l = f.readlines()
    for i in range(len(l)):
      if int(l[i])==0:
        g1.remove_node(i)    
  return nx.convert_node_labels_to_integers(g1,0)      


def delete_mis(g,filename):  # удаляет независимосе множество
  g1 = g.copy()
  with open(filename,"r") as f:
    l = f.readlines()
    for i in range(len(l)):
      if int(l[i])==1:
        g1.remove_node(i)    
  return nx.convert_node_labels_to_integers(g1,0)      

# размерность, расстояние, номера вершин, по окрестностям которых 
# выполняется отсечение
 
r = bin_graph(10,6,trim=[0, 984, 730, 826, 1000, 718, 857])


#g = bin_graph(9,6,trim=[0,63])  
#print(r.number_of_nodes())
#print(r.number_of_edges())
#write_metis(r,"test.graph")

#!./redumis test.graph --output test.mis --time_limit=30

#g = r.copy()
g1 = r.copy()

print(g1.number_of_nodes())
print(g1.number_of_edges())

#write_cnf(g1,10,"test.cnf")

#!./kissat1 test.cnf

nmis = 0

283
15247


In [3]:
FILENAME = "test_11"
COLORSS = 0

def is_not_colorable(graph, color_count, timer):
    global FILENAME, COLORSS
    COLORSS += 1
    coloring_filename = FILENAME + ".cnf"
    try:
        graph_copy = graph.copy()
        write_cnf(graph_copy, color_count, coloring_filename)
    except Exception as err:
        return True

    outputt = !timeout $timer ./kissat1 $coloring_filename
    if any(["UNSATISFIABLE" in line for line in outputt]):
         return True
    return False

def is_colorable(graph, color_count, timer):
    global FILENAME, COLORSS
    COLORSS += 1
    coloring_filename = FILENAME + ".cnf"
    try:
        graph_copy = graph.copy()
        write_cnf(graph_copy, color_count, coloring_filename)
    except Exception as err:
        return False

    outputt = !timeout $timer ./kissat1 $coloring_filename
    if all(["UNSATISFIABLE" not in line for line in outputt]):
        if any(["SATISFIABLE" in line for line in outputt]):
            return True
    return False

def check_colorable_fast(graph, max_colors=100, timer=120):
    if graph.number_of_edges() == 0:
        return (None, 0)

    max_uncolorable = None
    min_colorable = None
    
    left_border = 1
    right_border = max_colors + 1
    while right_border - left_border > 1:
        color_count_to_check = (right_border + left_border) // 2
        if is_not_colorable(graph, color_count_to_check, timer):
            left_border = color_count_to_check
        else:
            right_border = color_count_to_check
    max_uncolorable = left_border

    left_border = 1
    right_border = max_colors
    while right_border - left_border > 1:
        color_count_to_check = (right_border + left_border) // 2
        if is_colorable(graph, color_count_to_check, timer):
            right_border = color_count_to_check
        else:
            left_border = color_count_to_check
    if right_border == max_colors:
        if is_colorable(graph, max_colors, timer):
            min_colorable = right_border 
        return (max_uncolorable, min_colorable)
    else:
        min_colorable = right_border
        return (max_uncolorable, min_colorable)
    return (None, None)

# my_graph = small_graph(10, 6)
# qq = my_graph.copy()
# check_colorable_fast(qq)

# Ниже рабочая зона (Да)#


In [4]:
kv = [0, 945, 910, 630, 219, 365]
k_6 = []
k_4 = []
k_2 = []
for i in range(1024):
    if dist(i, 0) % 2 == 1 or dist(i, 0) > 6 or i in kv:
        continue
    flag = True
    for v in kv:
        if dist(v, i) > 6:
            flag = False
    if not flag:
        continue
    if dist(i, 0) == 6:
        k_6.append(i)
    elif dist(i, 0) == 4:
        k_4.append(i)
    elif dist(i, 0) == 2:
        k_2.append(i)
print(len(k_6), len(k_4), len(k_2))

130 85 15


In [5]:
k_24 = []
k_444 = []
for i in range(1024):
    if dist(i, 0) % 2 == 1 or dist(i, 0) > 6 or i in kv:
        continue
    flag = True
    for v in kv:
        if dist(v, i) > 6:
            flag = False
    if not flag:
        continue
    if dist(i, 0) < 6:
        continue
    flag = True
    for v in kv:
        if dist(v, i) == 2:
            flag = False
    if flag:
        k_444.append(i)
    else:
        k_24.append(i)

In [6]:
for i in k_24:
    if dist(i, 219) == 2 and dist(i, 365) == 4:
        print(bin(i))

print(bin(219))
print(bin(365))

0b11011101
0b11101011
0b101011011
0b11011011
0b101101101


In [7]:
k_24[0]

159

In [ ]:
from itertools import permutations
columns_perm = [0, 1, 2, 3]
for prm in permutations(columns_perm):
    print(type(prm))
    print(prm)

In [ ]:


def is_eq(old_group, base, new_v):
    columns_perm = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    vals_perm = [0, 1, 2, 3, 4, 5]
    flag = False
    
    for prm in permutations(columns_perm):
        old_group_after_perm = []
        for val in old_group:
            
        for vlv in permutations(vals_perm):
            old_group_after_perm = 

In [ ]:
specifics_list = []

for v in k_24:
    flag = True
    
    for lst in specifics_list:
        if is_eq(lst, kv, v):
            flag = False
    if flag:
        rar = kv + [v]
        specifics_list.append(rar) 

### Ниже Мусор ###

In [76]:
FILENAME = "ISK6"
kv = [0, 945, 910, 630, 219, 365]
k_6 = []
k_4 = []
k_2 = []
for i in range(1024):
    if dist(i, 0) % 2 == 1 or dist(i, 0) > 6 or i in kv:
        continue
    flag = True
    for v in kv:
        if dist(v, i) > 6:
            flag = False
    if not flag:
        continue
    if dist(i, 0) == 6:
        k_6.append(i)
    elif dist(i, 0) == 4:
        k_4.append(i)
    elif dist(i, 0) == 2:
        k_2.append(i)
print(len(k_6), len(k_4), len(k_2))

130 85 15


In [7]:
base_conf = kv

In [14]:
BIG_NUM = 3
COUNTER = 0
BPM = [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]

def simple_cook_graph_K6(base_conf, add_conf, k_2, k_4, k_6, n=10, k=6):   # третий параметр - множество вершин
    global COUNTER, BPM
    COUNTER += 1
    if COUNTER % 50 == 0:
        print(BPM)
    g = nx.Graph() 
    bans = base_conf + add_conf
    pooles = k_2 + k_4 + k_6
    for i in bans:
        g.add_node(i)
    
    for i in pooles:
        flag = True
        for j in bans:
            if dist(i,j) > k or dist(i, j) == 0:
                flag = False
                break
        if not flag:
            continue
        g.add_node(i)
    for i in list(g.nodes()):
        for j in g.nodes():
            if (i<j) and (dist(i,j)==k):
                g.add_edge(i,j)
    return nx.convert_node_labels_to_integers(g)

def sort_to_check(to_check, base_conf, add_conf):
    to_to = [[a, 0] for a in to_check]
    for a in to_to:
        for b in base_conf:
            if dist(a[0], b) == 6:
                a[1] += 1
        for b in  add_conf:
            if dist(a[0], b) == 6:
                a[1] += 1
    tbd = sorted(to_to, key=lambda x: (-x[1], x[0]))
    return [a[0] for a in tbd]


def bruteforce_nolimits_k6(to_check, base_conf, add_conf, k_2, k_4, how_much=BIG_NUM, timer=120, n=10, k=6, fromm=0):
    global COUNTER, BPM
    to_check = sort_to_check(to_check, base_conf, add_conf)
    if how_much == BIG_NUM:
        to_check = to_check[fromm:]
    ln = len(to_check)
    for i in range(ln):
        BPM[how_much][0] = i
        BPM[how_much][1] = ln
        # if i % 20 == 0 and how_much == BIG_NUM - 1:
        #     print(i, len(to_check))
        elem = to_check[i]
        if elem in base_conf or elem in add_conf:
            continue
        flag = True
        for el in add_conf:
            if dist(elem, el) > 6:
                flag = False
                break
        if not flag:
            continue

        if how_much > 1:
            bruteforce_nolimits_k6(to_check[i+1:], base_conf, add_conf + [elem], k_2=k_2,
                                     k_4=k_4, 
                                     how_much=how_much-1,timer=timer, n=n, k=k)
        else:
            grph = simple_cook_graph_K6(base_conf=base_conf, add_conf=add_conf + [elem], k_2=k_2, k_4=k_4, k_6=to_check[i+1:], n=n, k=k)
            brb = is_colorable(grph, 11, timer)
            if not brb:
                print(base_conf, add_conf + [elem])
    if how_much == BIG_NUM:
        print("analysing {} finished".format(BIG_NUM))

def bruteforce_nolimits_k6_short(to_check, base_conf, add_conf, k_2, k_4, how_much=4, timer=120, n=10, k=6, fromm=0):
    global COUNTER, BPM
    to_check = sort_to_check(to_check, base_conf, add_conf)
    if how_much == BIG_NUM:
        to_check = to_check[fromm:]
    ln = len(to_check)
    for i in range(ln):
        BPM[how_much][0] = i
        BPM[how_much][1] = ln
        elem = to_check[i]
        if elem in base_conf or elem in add_conf:
            continue
        flag = True
        for el in add_conf:
            if dist(elem, el) > 6:
                flag = False
                break
        if not flag:
            continue
    
        grph = simple_cook_graph_K6(base_conf=base_conf, add_conf=add_conf + [elem], k_2=k_2, k_4=k_4, k_6=[], n=n, k=k)
        brb = is_colorable(grph, 11, timer)
        if not brb:
            if how_much == 1:
                print(base_conf, add_conf + [elem])
        if how_much > 1:
            bruteforce_nolimits_k6_short(to_check[i+1:], base_conf, add_conf + [elem], k_2=k_2,
                                     k_4=k_4, how_much=how_much-1,timer=timer, n=n, k=k)

    if how_much == BIG_NUM:
        print("analysing {}-or-less-but-at-least-1 finished".format(BIG_NUM))

In [64]:
BIG_NUM=5

In [65]:
k_6 = sort_to_check(k_6_6, base_conf, [])

In [59]:
# k_2 = k_6_6[:60] + k_2
# k_6 = k_6_6[60:]

In [77]:
%%time
BIG_NUM=5
print("Before 1-COLORED", COLORSS)
bruteforce_nolimits_k6_short(k_6, base_conf, [], k_2, k_4, BIG_NUM - 4, 120, 10, 6)
print("1 finished")
print("AFTER 1-COLORED", COLORSS)

Before 1-COLORED 28330
[[0, 70], [5, 130], [0, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [55, 130], [0, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [105, 130], [0, 130], [0, 130], [0, 130], [0, 80]]
1 finished
AFTER 1-COLORED 28460
CPU times: user 1min 5s, sys: 5.09 s, total: 1min 10s
Wall time: 1min 20s


In [78]:
%%time
bruteforce_nolimits_k6_short(k_6, base_conf, [], k_2, k_4, BIG_NUM - 3, 120, 10, 6)
print("2 finished")
print("AFTER 2-COLORED", COLORSS)

[[0, 70], [24, 129], [0, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [74, 129], [0, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [3, 128], [1, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [53, 128], [1, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [107, 128], [1, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [33, 127], [2, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [87, 127], [2, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [14, 126], [3, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [64, 126], [3, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [122, 126], [3, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [46, 125], [4, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [100, 125], [4, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [29, 124], [5, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [83, 124], [5, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [13, 123], [6, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [66, 123], [6, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [122, 123], [6, 130], [0, 13

[[0, 70], [43, 46], [83, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [8, 44], [85, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [26, 43], [86, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [41, 42], [87, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [8, 40], [89, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [19, 39], [90, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [31, 38], [91, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [7, 36], [93, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [21, 35], [94, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [8, 33], [96, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [27, 32], [97, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [20, 30], [99, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [11, 28], [101, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [9, 26], [103, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [11, 24], [105, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [19, 22], [107, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [6, 19], [110, 130], [0, 130], 

In [79]:
%%time
bruteforce_nolimits_k6_short(k_6, base_conf, [], k_2, k_4, BIG_NUM - 2, 120, 10, 6)
print("3 finished")
print("AFTER 3-COLORED", COLORSS)

[[0, 70], [0, 1], [129, 130], [0, 130], [0, 130], [0, 80]]
[[0, 70], [50, 128], [0, 129], [0, 130], [0, 130], [0, 80]]
[[0, 70], [110, 128], [0, 129], [0, 130], [0, 130], [0, 80]]
[[0, 70], [38, 127], [1, 129], [0, 130], [0, 130], [0, 80]]
[[0, 70], [90, 127], [1, 129], [0, 130], [0, 130], [0, 80]]
[[0, 70], [27, 126], [2, 129], [0, 130], [0, 130], [0, 80]]
[[0, 70], [83, 126], [2, 129], [0, 130], [0, 130], [0, 80]]
[[0, 70], [18, 125], [3, 129], [0, 130], [0, 130], [0, 80]]
[[0, 70], [70, 125], [3, 129], [0, 130], [0, 130], [0, 80]]
[[0, 70], [10, 124], [4, 129], [0, 130], [0, 130], [0, 80]]
[[0, 70], [62, 124], [4, 129], [0, 130], [0, 130], [0, 80]]
[[0, 70], [3, 123], [5, 129], [0, 130], [0, 130], [0, 80]]
[[0, 70], [55, 123], [5, 129], [0, 130], [0, 130], [0, 80]]
[[0, 70], [115, 123], [5, 129], [0, 130], [0, 130], [0, 80]]
[[0, 70], [49, 122], [6, 129], [0, 130], [0, 130], [0, 80]]
[[0, 70], [112, 122], [6, 129], [0, 130], [0, 130], [0, 80]]
[[0, 70], [44, 121], [7, 129], [0, 130]

[[0, 70], [73, 126], [1, 128], [1, 130], [0, 130], [0, 80]]
[[0, 70], [12, 125], [2, 128], [1, 130], [0, 130], [0, 80]]
[[0, 70], [64, 125], [2, 128], [1, 130], [0, 130], [0, 80]]
[[0, 70], [4, 124], [3, 128], [1, 130], [0, 130], [0, 80]]
[[0, 70], [56, 124], [3, 128], [1, 130], [0, 130], [0, 80]]
[[0, 70], [119, 124], [3, 128], [1, 130], [0, 130], [0, 80]]
[[0, 70], [49, 123], [4, 128], [1, 130], [0, 130], [0, 80]]
[[0, 70], [107, 123], [4, 128], [1, 130], [0, 130], [0, 80]]
[[0, 70], [43, 122], [5, 128], [1, 130], [0, 130], [0, 80]]
[[0, 70], [103, 122], [5, 128], [1, 130], [0, 130], [0, 80]]
[[0, 70], [37, 121], [6, 128], [1, 130], [0, 130], [0, 80]]
[[0, 70], [90, 121], [6, 128], [1, 130], [0, 130], [0, 80]]
[[0, 70], [31, 120], [7, 128], [1, 130], [0, 130], [0, 80]]
[[0, 70], [90, 120], [7, 128], [1, 130], [0, 130], [0, 80]]
[[0, 70], [29, 119], [8, 128], [1, 130], [0, 130], [0, 80]]
[[0, 70], [86, 119], [8, 128], [1, 130], [0, 130], [0, 80]]
[[0, 70], [26, 118], [9, 128], [1, 130

[[0, 70], [47, 122], [4, 127], [2, 130], [0, 130], [0, 80]]
[[0, 70], [105, 122], [4, 127], [2, 130], [0, 130], [0, 80]]
[[0, 70], [42, 121], [5, 127], [2, 130], [0, 130], [0, 80]]
[[0, 70], [102, 121], [5, 127], [2, 130], [0, 130], [0, 80]]
[[0, 70], [38, 120], [6, 127], [2, 130], [0, 130], [0, 80]]
[[0, 70], [94, 120], [6, 127], [2, 130], [0, 130], [0, 80]]
[[0, 70], [35, 119], [7, 127], [2, 130], [0, 130], [0, 80]]
[[0, 70], [93, 119], [7, 127], [2, 130], [0, 130], [0, 80]]
[[0, 70], [31, 118], [8, 127], [2, 130], [0, 130], [0, 80]]
[[0, 70], [84, 118], [8, 127], [2, 130], [0, 130], [0, 80]]
[[0, 70], [32, 117], [9, 127], [2, 130], [0, 130], [0, 80]]
[[0, 70], [86, 117], [9, 127], [2, 130], [0, 130], [0, 80]]
[[0, 70], [32, 116], [10, 127], [2, 130], [0, 130], [0, 80]]
[[0, 70], [90, 116], [10, 127], [2, 130], [0, 130], [0, 80]]
[[0, 70], [31, 115], [11, 127], [2, 130], [0, 130], [0, 80]]
[[0, 70], [90, 115], [11, 127], [2, 130], [0, 130], [0, 80]]
[[0, 70], [35, 114], [12, 127], [2

[[0, 70], [51, 117], [8, 126], [3, 130], [0, 130], [0, 80]]
[[0, 70], [111, 117], [8, 126], [3, 130], [0, 130], [0, 80]]
[[0, 70], [51, 116], [9, 126], [3, 130], [0, 130], [0, 80]]
[[0, 70], [114, 116], [9, 126], [3, 130], [0, 130], [0, 80]]
[[0, 70], [53, 115], [10, 126], [3, 130], [0, 130], [0, 80]]
[[0, 70], [0, 114], [11, 126], [3, 130], [0, 130], [0, 80]]
[[0, 70], [56, 114], [11, 126], [3, 130], [0, 130], [0, 80]]
[[0, 70], [3, 113], [12, 126], [3, 130], [0, 130], [0, 80]]
[[0, 70], [58, 113], [12, 126], [3, 130], [0, 130], [0, 80]]
[[0, 70], [7, 112], [13, 126], [3, 130], [0, 130], [0, 80]]
[[0, 70], [63, 112], [13, 126], [3, 130], [0, 130], [0, 80]]
[[0, 70], [12, 111], [14, 126], [3, 130], [0, 130], [0, 80]]
[[0, 70], [65, 111], [14, 126], [3, 130], [0, 130], [0, 80]]
[[0, 70], [16, 110], [15, 126], [3, 130], [0, 130], [0, 80]]
[[0, 70], [74, 110], [15, 126], [3, 130], [0, 130], [0, 80]]
[[0, 70], [22, 109], [16, 126], [3, 130], [0, 130], [0, 80]]
[[0, 70], [77, 109], [16, 126

[[0, 70], [107, 111], [13, 125], [4, 130], [0, 130], [0, 80]]
[[0, 70], [49, 110], [14, 125], [4, 130], [0, 130], [0, 80]]
[[0, 70], [1, 109], [15, 125], [4, 130], [0, 130], [0, 80]]
[[0, 70], [55, 109], [15, 125], [4, 130], [0, 130], [0, 80]]
[[0, 70], [8, 108], [16, 125], [4, 130], [0, 130], [0, 80]]
[[0, 70], [63, 108], [16, 125], [4, 130], [0, 130], [0, 80]]
[[0, 70], [15, 107], [17, 125], [4, 130], [0, 130], [0, 80]]
[[0, 70], [73, 107], [17, 125], [4, 130], [0, 130], [0, 80]]
[[0, 70], [24, 106], [18, 125], [4, 130], [0, 130], [0, 80]]
[[0, 70], [84, 106], [18, 125], [4, 130], [0, 130], [0, 80]]
[[0, 70], [32, 105], [19, 125], [4, 130], [0, 130], [0, 80]]
[[0, 70], [93, 105], [19, 125], [4, 130], [0, 130], [0, 80]]
[[0, 70], [43, 104], [20, 125], [4, 130], [0, 130], [0, 80]]
[[0, 70], [1, 103], [21, 125], [4, 130], [0, 130], [0, 80]]
[[0, 70], [56, 103], [21, 125], [4, 130], [0, 130], [0, 80]]
[[0, 70], [14, 102], [22, 125], [4, 130], [0, 130], [0, 80]]
[[0, 70], [74, 102], [22, 

...

[[0, 70], [9, 18], [45, 64], [65, 130], [0, 130], [0, 80]]
[[0, 70], [2, 5], [58, 64], [65, 130], [0, 130], [0, 80]]
[[0, 70], [43, 62], [0, 63], [66, 130], [0, 130], [0, 80]]
[[0, 70], [44, 61], [1, 63], [66, 130], [0, 130], [0, 80]]
[[0, 70], [1, 59], [3, 63], [66, 130], [0, 130], [0, 80]]
[[0, 70], [7, 58], [4, 63], [66, 130], [0, 130], [0, 80]]
[[0, 70], [14, 57], [5, 63], [66, 130], [0, 130], [0, 80]]
[[0, 70], [23, 56], [6, 63], [66, 130], [0, 130], [0, 80]]
[[0, 70], [33, 55], [7, 63], [66, 130], [0, 130], [0, 80]]
[[0, 70], [53, 54], [9, 63], [66, 130], [0, 130], [0, 80]]
[[0, 70], [11, 52], [10, 63], [66, 130], [0, 130], [0, 80]]
[[0, 70], [24, 51], [11, 63], [66, 130], [0, 130], [0, 80]]
[[0, 70], [1, 49], [13, 63], [66, 130], [0, 130], [0, 80]]
[[0, 70], [19, 48], [14, 63], [66, 130], [0, 130], [0, 80]]
[[0, 70], [38, 47], [15, 63], [66, 130], [0, 130], [0, 80]]
[[0, 70], [14, 45], [17, 63], [66, 130], [0, 130], [0, 80]]
[[0, 70], [40, 44], [18, 63], [66, 130], [0, 130], [0,

[[0, 70], [22, 44], [13, 58], [71, 130], [0, 130], [0, 80]]
[[0, 70], [7, 42], [15, 58], [71, 130], [0, 130], [0, 80]]
[[0, 70], [37, 41], [16, 58], [71, 130], [0, 130], [0, 80]]
[[0, 70], [20, 39], [18, 58], [71, 130], [0, 130], [0, 80]]
[[0, 70], [12, 37], [20, 58], [71, 130], [0, 130], [0, 80]]
[[0, 70], [3, 35], [22, 58], [71, 130], [0, 130], [0, 80]]
[[0, 70], [9, 33], [24, 58], [71, 130], [0, 130], [0, 80]]
[[0, 70], [7, 31], [26, 58], [71, 130], [0, 130], [0, 80]]
[[0, 70], [10, 29], [28, 58], [71, 130], [0, 130], [0, 80]]
[[0, 70], [9, 26], [31, 58], [71, 130], [0, 130], [0, 80]]
[[0, 70], [7, 20], [37, 58], [71, 130], [0, 130], [0, 80]]
[[0, 70], [11, 13], [44, 58], [71, 130], [0, 130], [0, 80]]
[[0, 70], [6, 8], [49, 58], [71, 130], [0, 130], [0, 80]]
[[0, 70], [23, 56], [0, 57], [72, 130], [0, 130], [0, 80]]
[[0, 70], [32, 55], [1, 57], [72, 130], [0, 130], [0, 80]]
[[0, 70], [49, 54], [2, 57], [72, 130], [0, 130], [0, 80]]
[[0, 70], [8, 52], [4, 57], [72, 130], [0, 130], [0

[[0, 70], [24, 42], [8, 51], [78, 130], [0, 130], [0, 80]]
[[0, 70], [3, 40], [10, 51], [78, 130], [0, 130], [0, 80]]
[[0, 70], [27, 39], [11, 51], [78, 130], [0, 130], [0, 80]]
[[0, 70], [12, 37], [13, 51], [78, 130], [0, 130], [0, 80]]
[[0, 70], [11, 35], [15, 51], [78, 130], [0, 130], [0, 80]]
[[0, 70], [6, 33], [17, 51], [78, 130], [0, 130], [0, 80]]
[[0, 70], [7, 31], [19, 51], [78, 130], [0, 130], [0, 80]]
[[0, 70], [10, 29], [21, 51], [78, 130], [0, 130], [0, 80]]
[[0, 70], [11, 27], [23, 51], [78, 130], [0, 130], [0, 80]]
[[0, 70], [24, 25], [26, 51], [78, 130], [0, 130], [0, 80]]
[[0, 70], [2, 19], [31, 51], [78, 130], [0, 130], [0, 80]]
[[0, 70], [7, 15], [35, 51], [78, 130], [0, 130], [0, 80]]
[[0, 70], [0, 8], [42, 51], [78, 130], [0, 130], [0, 80]]
[[0, 70], [17, 49], [0, 50], [79, 130], [0, 130], [0, 80]]
[[0, 70], [44, 48], [1, 50], [79, 130], [0, 130], [0, 80]]
[[0, 70], [11, 46], [3, 50], [79, 130], [0, 130], [0, 80]]
[[0, 70], [38, 45], [4, 50], [79, 130], [0, 130], [

[[0, 70], [39, 40], [1, 41], [88, 130], [0, 130], [0, 80]]
[[0, 70], [14, 38], [2, 41], [88, 130], [0, 130], [0, 80]]
[[0, 70], [30, 37], [3, 41], [88, 130], [0, 130], [0, 80]]
[[0, 70], [13, 35], [5, 41], [88, 130], [0, 130], [0, 80]]
[[0, 70], [1, 33], [7, 41], [88, 130], [0, 130], [0, 80]]
[[0, 70], [23, 32], [8, 41], [88, 130], [0, 130], [0, 80]]
[[0, 70], [15, 30], [10, 41], [88, 130], [0, 130], [0, 80]]
[[0, 70], [16, 28], [12, 41], [88, 130], [0, 130], [0, 80]]
[[0, 70], [15, 26], [14, 41], [88, 130], [0, 130], [0, 80]]
[[0, 70], [15, 24], [16, 41], [88, 130], [0, 130], [0, 80]]
[[0, 70], [20, 22], [18, 41], [88, 130], [0, 130], [0, 80]]
[[0, 70], [15, 19], [21, 41], [88, 130], [0, 130], [0, 80]]
[[0, 70], [7, 15], [25, 41], [88, 130], [0, 130], [0, 80]]
[[0, 70], [9, 11], [29, 41], [88, 130], [0, 130], [0, 80]]
[[0, 70], [2, 39], [0, 40], [89, 130], [0, 130], [0, 80]]
[[0, 70], [17, 38], [1, 40], [89, 130], [0, 130], [0, 80]]
[[0, 70], [34, 37], [2, 40], [89, 130], [0, 130], [0

[[0, 70], [4, 18], [9, 28], [101, 130], [0, 130], [0, 80]]
[[0, 70], [0, 14], [13, 28], [101, 130], [0, 130], [0, 80]]
[[0, 70], [3, 7], [20, 28], [101, 130], [0, 130], [0, 80]]
[[0, 70], [0, 25], [1, 27], [102, 130], [0, 130], [0, 80]]
[[0, 70], [11, 23], [3, 27], [102, 130], [0, 130], [0, 80]]
[[0, 70], [0, 20], [6, 27], [102, 130], [0, 130], [0, 80]]
[[0, 70], [16, 18], [8, 27], [102, 130], [0, 130], [0, 80]]
[[0, 70], [4, 14], [12, 27], [102, 130], [0, 130], [0, 80]]
[[0, 70], [1, 8], [18, 27], [102, 130], [0, 130], [0, 80]]
[[0, 70], [19, 25], [0, 26], [103, 130], [0, 130], [0, 80]]
[[0, 70], [4, 22], [3, 26], [103, 130], [0, 130], [0, 80]]
[[0, 70], [17, 20], [5, 26], [103, 130], [0, 130], [0, 80]]
[[0, 70], [14, 17], [8, 26], [103, 130], [0, 130], [0, 80]]
[[0, 70], [5, 13], [12, 26], [103, 130], [0, 130], [0, 80]]
[[0, 70], [6, 7], [20, 26], [103, 130], [0, 130], [0, 80]]
[[0, 70], [10, 23], [1, 25], [104, 130], [0, 130], [0, 80]]
[[0, 70], [3, 20], [4, 25], [104, 130], [0, 130

In [80]:
%%time
bruteforce_nolimits_k6_short(k_6, base_conf, [], k_2, k_4, BIG_NUM - 1, 120, 10, 6)
print("4 finished")
print("AFTER 4-COLORED", COLORSS)

[[0, 70], [2, 127], [0, 128], [0, 129], [0, 130], [0, 80]]
[[0, 70], [53, 127], [0, 128], [0, 129], [0, 130], [0, 80]]
[[0, 70], [1, 126], [1, 128], [0, 129], [0, 130], [0, 80]]
[[0, 70], [51, 126], [1, 128], [0, 129], [0, 130], [0, 80]]
[[0, 70], [1, 125], [2, 128], [0, 129], [0, 130], [0, 80]]
[[0, 70], [52, 125], [2, 128], [0, 129], [0, 130], [0, 80]]
[[0, 70], [2, 124], [3, 128], [0, 129], [0, 130], [0, 80]]
[[0, 70], [53, 124], [3, 128], [0, 129], [0, 130], [0, 80]]
[[0, 70], [4, 123], [4, 128], [0, 129], [0, 130], [0, 80]]
[[0, 70], [57, 123], [4, 128], [0, 129], [0, 130], [0, 80]]
[[0, 70], [7, 122], [5, 128], [0, 129], [0, 130], [0, 80]]
[[0, 70], [62, 122], [5, 128], [0, 129], [0, 130], [0, 80]]
[[0, 70], [12, 121], [6, 128], [0, 129], [0, 130], [0, 80]]
[[0, 70], [67, 121], [6, 128], [0, 129], [0, 130], [0, 80]]
[[0, 70], [16, 120], [7, 128], [0, 129], [0, 130], [0, 80]]
[[0, 70], [73, 120], [7, 128], [0, 129], [0, 130], [0, 80]]
[[0, 70], [23, 119], [8, 128], [0, 129], [0, 1

[[0, 70], [14, 113], [13, 127], [1, 129], [0, 130], [0, 80]]
[[0, 70], [70, 113], [13, 127], [1, 129], [0, 130], [0, 80]]
[[0, 70], [28, 112], [14, 127], [1, 129], [0, 130], [0, 80]]
[[0, 70], [89, 112], [14, 127], [1, 129], [0, 130], [0, 80]]
[[0, 70], [47, 111], [15, 127], [1, 129], [0, 130], [0, 80]]
[[0, 70], [5, 110], [16, 127], [1, 129], [0, 130], [0, 80]]
[[0, 70], [59, 110], [16, 127], [1, 129], [0, 130], [0, 80]]
[[0, 70], [25, 109], [17, 127], [1, 129], [0, 130], [0, 80]]
[[0, 70], [93, 109], [17, 127], [1, 129], [0, 130], [0, 80]]
[[0, 70], [40, 108], [18, 127], [1, 129], [0, 130], [0, 80]]
[[0, 70], [3, 107], [19, 127], [1, 129], [0, 130], [0, 80]]
[[0, 70], [61, 107], [19, 127], [1, 129], [0, 130], [0, 80]]
[[0, 70], [20, 106], [20, 127], [1, 129], [0, 130], [0, 80]]
[[0, 70], [82, 106], [20, 127], [1, 129], [0, 130], [0, 80]]
[[0, 70], [38, 105], [21, 127], [1, 129], [0, 130], [0, 80]]
[[0, 70], [6, 104], [22, 127], [1, 129], [0, 130], [0, 80]]
[[0, 70], [66, 104], [22, 1

[[0, 70], [82, 95], [30, 126], [2, 129], [0, 130], [0, 80]]
[[0, 70], [48, 94], [31, 126], [2, 129], [0, 130], [0, 80]]
[[0, 70], [30, 93], [32, 126], [2, 129], [0, 130], [0, 80]]
[[0, 70], [2, 92], [33, 126], [2, 129], [0, 130], [0, 80]]
[[0, 70], [67, 92], [33, 126], [2, 129], [0, 130], [0, 80]]
[[0, 70], [45, 91], [34, 126], [2, 129], [0, 130], [0, 80]]
[[0, 70], [15, 90], [35, 126], [2, 129], [0, 130], [0, 80]]
[[0, 70], [2, 88], [37, 126], [2, 129], [0, 130], [0, 80]]
[[0, 70], [64, 88], [37, 126], [2, 129], [0, 130], [0, 80]]
[[0, 70], [41, 87], [38, 126], [2, 129], [0, 130], [0, 80]]
[[0, 70], [17, 86], [39, 126], [2, 129], [0, 130], [0, 80]]
[[0, 70], [83, 86], [39, 126], [2, 129], [0, 130], [0, 80]]
[[0, 70], [61, 85], [40, 126], [2, 129], [0, 130], [0, 80]]
[[0, 70], [48, 84], [41, 126], [2, 129], [0, 130], [0, 80]]
[[0, 70], [36, 83], [42, 126], [2, 129], [0, 130], [0, 80]]
[[0, 70], [24, 82], [43, 126], [2, 129], [0, 130], [0, 80]]
[[0, 70], [15, 80], [45, 126], [2, 129], [

[[0, 70], [57, 67], [57, 125], [3, 129], [0, 130], [0, 80]]
[[0, 70], [56, 66], [58, 125], [3, 129], [0, 130], [0, 80]]
[[0, 70], [64, 65], [60, 125], [3, 129], [0, 130], [0, 80]]
[[0, 70], [1, 63], [61, 125], [3, 129], [0, 130], [0, 80]]
[[0, 70], [14, 62], [62, 125], [3, 129], [0, 130], [0, 80]]
[[0, 70], [18, 61], [63, 125], [3, 129], [0, 130], [0, 80]]
[[0, 70], [44, 60], [64, 125], [3, 129], [0, 130], [0, 80]]
[[0, 70], [5, 58], [66, 125], [3, 129], [0, 130], [0, 80]]
[[0, 70], [23, 57], [67, 125], [3, 129], [0, 130], [0, 80]]
[[0, 70], [36, 56], [68, 125], [3, 129], [0, 130], [0, 80]]
[[0, 70], [0, 54], [70, 125], [3, 129], [0, 130], [0, 80]]
[[0, 70], [24, 53], [71, 125], [3, 129], [0, 130], [0, 80]]
[[0, 70], [39, 51], [73, 125], [3, 129], [0, 130], [0, 80]]
[[0, 70], [11, 48], [76, 125], [3, 129], [0, 130], [0, 80]]
[[0, 70], [46, 47], [79, 125], [3, 129], [0, 130], [0, 80]]
[[0, 70], [35, 44], [80, 125], [3, 129], [0, 130], [0, 80]]
[[0, 70], [18, 42], [82, 125], [3, 129], [0

KeyboardInterrupt: 

In [66]:
# %%time
# BIG_NUM=5
# bruteforce_nolimits_k6_short(k_6, base_conf, [], k_2, k_4, BIG_NUM - 1, 120, 10, 6)
# print("4 finished")
# print("AFTER 4-COLORED", COLORSS)
# bruteforce_nolimits_k6_short(k_6, base_conf, [], k_2, k_4, BIG_NUM - 2, 120, 10, 6)
# print("3 finished")
# print("AFTER 3-COLORED", COLORSS)
# bruteforce_nolimits_k6_short(k_6, base_conf, [], k_2, k_4, BIG_NUM - 3, 120, 10, 6)
# print("2 finished")
# print("AFTER 2-COLORED", COLORSS)
# bruteforce_nolimits_k6_short(k_6, base_conf, [], k_2, k_4, BIG_NUM - 4, 120, 10, 6)
# print("1 finished")
# print("AFTER 1-COLORED", COLORSS)

[0, 945, 910, 630, 219, 365] [159, 359, 634, 221]
[[0, 70], [11, 127], [0, 128], [0, 129], [0, 130], [0, 80]]
[0, 945, 910, 630, 219, 365] [159, 359, 634, 938]
[0, 945, 910, 630, 219, 365] [159, 359, 634, 435]
[0, 945, 910, 630, 219, 365] [159, 359, 634, 483]
[0, 945, 910, 630, 219, 365] [159, 359, 634, 574]
^C
[0, 945, 910, 630, 219, 365] [159, 359, 634, 603]



KeyboardInterrupt



In [70]:
print("we win")

we win


In [67]:
%%time
grph = simple_cook_graph_K6(base_conf=base_conf,
                        add_conf=[],
                        k_2=k_2,
                        k_4=k_4,
                        k_6=[], n=10, k=6)
brb = is_colorable(grph, 11, 20)
brb

KeyboardInterrupt: 

In [71]:
%%time
grph = simple_cook_graph_K6(base_conf=base_conf,
                        add_conf=[159, 359, 634, 938],
                        k_2=k_2,
                        k_4=k_4,
                        k_6=[], n=10, k=6)
brb = is_colorable(grph, 11, 30)
brb

CPU times: user 1.81 s, sys: 101 ms, total: 1.91 s
Wall time: 31.9 s


False

In [68]:
print(COLORSS)

28318


In [69]:
%%time
print("COLORED", COLORSS)
k_6 = sort_to_check(k_6_6, base_conf, [])
print('to check:', len(k_6), len(k_6))
bruteforce_nolimits_k6(k_6, base_conf, [], k_2, k_4, BIG_NUM, 20, 10, 6)
print("AFTER 3-COLORED", COLORSS)

COLORED 28318



KeyboardInterrupt

